# REAM-MoE Quickstart Guide

This notebook will guide you through compressing a Mixture-of-Experts (MoE) model using the REAM-MoE library.

## What is REAM-MoE?

REAM-MoE compresses large language models with MoE architectures by:
- **Expert Pruning**: Removing low-importance experts
- **Expert Merging**: Combining similar experts using permutation-aware averaging

This reduces model size and inference cost while preserving most of the model's capabilities.

## Installation

First, install the library:

In [ ]:
# Install the library (if needed)
# !pip install -e .

# Or install from requirements
# !pip install -r requirements.txt

## Setup

Import the necessary libraries and configure logging:

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Import REAM-MoE components
from ream_moe import (
    observe_model,
    prune_model,
    merge_model,
    PruningConfig,
    MergeConfig,
    verify_model_config,
    print_verification_result,
    setup_logging,
)
from ream_moe.calibration import build_calibration_batches, list_available_datasets

# Optional: suppress INFO logs to reduce noise
setup_logging(level="WARNING")

## 1. Load Model

Load your MoE model and tokenizer. This example uses Qwen3-14B-MoE, but you can use any supported model family.

In [ ]:
# Model configuration
MODEL_NAME = "Qwen/Qwen3-14B-MoE"  # Change to your model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
)

# Set padding token if not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

print(f"Loaded model: {model.__class__.__name__}")
print(f"Device: {DEVICE}")

## 2. Verify Model Configuration

Check if your model is properly supported by REAM-MoE:

In [ ]:
# Verify the model configuration
result = verify_model_config(MODEL_NAME, model)
print_verification_result(result)

if not result["valid"]:
    print("\nWarning: Model verification failed. You may need to add configuration for this model.")

## 3. Prepare Calibration Data

Build calibration batches to collect activation statistics. You can use built-in datasets or your own texts.

In [ ]:
# List available built-in datasets
print("Available calibration datasets:")
for ds in list_available_datasets():
    print(f"  - {ds}")

In [ ]:
# Build calibration batches
# Options: "c4", "code", "math", "writing", "hardcoded", "combined"
DATASET = "hardcoded"  # Recommended - uses diverse hardcoded prompts
SAMPLES = 2000  # Number of samples for calibration
MAX_SEQ_LEN = 512
BATCH_SIZE = 4

batches = list(build_calibration_batches(
    tokenizer,
    DATASET,
    max_seq_len=MAX_SEQ_LEN,
    batch_size=BATCH_SIZE,
    samples=SAMPLES,
))

print(f"Created {len(batches)} calibration batches")
print(f"Total samples: {len(batches) * BATCH_SIZE}")

## 4. Collect Activation Statistics

Run the model on calibration data to collect expert activation statistics:

In [ ]:
# Collect observer data
observer_data = {}

model.eval()
with torch.no_grad():
    for i, batch in enumerate(batches):
        if i >= 100:  # Limit batches for quick testing
            break
        
        # Get batch data
        input_ids = batch.input_ids.to(DEVICE)
        attention_mask = batch.attention_mask.to(DEVICE)
        
        # Collect statistics
        batch_data = observe_model(
            model,
            input_ids,
            attention_mask,
        )
        
        # Merge batch data
        for layer_idx, data in batch_data.items():
            if layer_idx not in observer_data:
                observer_data[layer_idx] = data
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(batches)} batches")

print(f"\nCollected statistics for {len(observer_data)} layers")

## 5. Compress the Model

### Option A: Expert Pruning

Remove low-importance experts to reduce model size:

In [ ]:
# Prune 25% of experts (keep 75%)
prune_config = PruningConfig(
    compression_ratio=0.25,  # Remove 25% of experts
    prune_method="saliency_scores",  # Use REAP saliency
    preserve_super_experts=False,  # Set to True to preserve high-activation experts
)

retained_counts = prune_model(model, observer_data, prune_config)

avg_experts = sum(retained_counts.values()) / len(retained_counts)
print(f"Average experts per layer after pruning: {avg_experts:.1f}")

### Option B: Expert Merging (Alternative)

Instead of pruning, you can merge similar experts:

In [ ]:
# Merge experts to keep 75% (25% compression)
# Uncomment to use merging instead of pruning
# merge_config = MergeConfig(
#     target_ratio=0.75,  # Keep 75% of experts
#     group_size=16,  # Max experts per group
# )
# retained_counts = merge_model(model, observer_data, merge_config)
# 
# avg_experts = sum(retained_counts.values()) / len(retained_counts)
# print(f"Average experts per layer after merging: {avg_experts:.1f}")

## 6. Save Compressed Model

Save the compressed model to disk:

In [ ]:
OUTPUT_DIR = "./compressed_model"

# Save model and tokenizer
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to: {OUTPUT_DIR}")
print(f"Compression info:")
print(f"  - Original experts: Check model.config")
print(f"  - Compressed experts: {avg_experts:.1f} per layer")
print(f"  - Compression ratio: ~{prune_config.compression_ratio:.0%}")

## 7. Test Compressed Model

Load and test the compressed model:

In [ ]:
# Load compressed model
compressed_model = AutoModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Test generation
prompt = "Explain the difference between supervised and unsupervised learning."
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    outputs = compressed_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Response:")
print(response)

## Advanced Options

### Using Custom Calibration Data

You can use your own texts for calibration:

In [ ]:
# Use custom texts
my_texts = [
    "Your custom text here...",
    "More text samples...",
    "Add as many as you want",
]

custom_batches = list(build_calibration_batches(
    tokenizer,
    my_texts,
    max_seq_len=512,
    batch_size=4,
))

print(f"Created {len(custom_batches)} custom calibration batches")

### Listing Supported Models

Check which model families are supported:

In [ ]:
from ream_moe import list_supported_models

print("Supported model families:")
for model_class in list_supported_models():
    print(f"  - {model_class}")

## Tips & Best Practices

1. **Calibration Data Quality**: Use diverse, representative text for calibration. The "hardcoded" dataset covers multiple domains.

2. **Compression Ratio**: Start with 0.25 (25% compression) and adjust based on quality vs. size tradeoff.

3. **Preserve Super Experts**: Enable `preserve_super_experts=True` if you notice important capabilities degrading.

4. **Verification**: Always verify your model configuration before compressing.

5. **Testing**: Thoroughly test your compressed model on your specific use cases.

## Troubleshooting

- **Model not supported**: Use `verify_model_config()` to check. The library may auto-detect unknown models.

- **Out of memory**: Reduce `batch_size`, `max_seq_len`, or `samples`. Use `store_on_cpu=True` in ObserverConfig.

- **Quality degradation**: Try lower compression ratio or enable `preserve_super_experts`.

## Next Steps

- Check the [README](../README.md) for CLI usage
- Explore different calibration datasets for your use case
- Experiment with both pruning and merging methods
- Contribute configurations for unsupported models